In [ ]:
%load_ext autoreload
%autoreload 2
import time
import multiprocessing
from pathlib import Path
import shutil
import numpy as np
import thuner.data as data
import thuner.data.dispatch as dispatch
import thuner.grid as grid
import thuner.option as option
import thuner.track as track
import thuner.analyze as analyze
import thuner.parallel as parallel
import thuner.visualize as visualize
import thuner.attribute as attribute
import thuner.log as log

notebook_name = "gridrad_demo.ipynb"

In [ ]:
# Parent directory for saving outputs
base_local = Path.home() / "THUNER_output"
year=2010
event_directories = data.gridrad.get_event_directories(year, base_local=base_local)
event_directory = event_directories[0]
start, end, event_start = data.gridrad.get_event_times(event_directory)
start = "2010-01-20T22:00:00"
end = "2010-01-21T02:00:00"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/gridrad_demo"
event_start

In [ ]:
# if output_parent.exists():
# shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create and save the dataset options
times_dict = {"start": start, "end": end}
gridrad_dict = {"event_start": event_start}
gridrad_options = data.gridrad.GridRadSevereOptions(**times_dict, **gridrad_dict)
era5_dict = {"latitude_range": [27, 39], "longitude_range": [-102, -89]}
era5_pl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
data_options = option.data.DataOptions(
    datasets=[gridrad_options, era5_pl_options, era5_sl_options]
)
data_options.to_yaml(options_directory / "data.yml")
gridrad_options = data_options.dataset_by_name("gridrad")

# Create and save the grid_options dictionary 
grid_options = option.grid.GridOptions(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options dictionary
track_options = option.default.track(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
# track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
# track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
# visualize_options = {
#     obj: visualize.option.runtime_options(obj, save=True, style="presentation")
#     for obj in ["mcs"]
# }
visualize_options = None

In [ ]:
times = data.utils.generate_times(data_options.dataset_by_name("gridrad"))
args = [times, data_options.model_copy(deep=True), grid_options.model_copy(deep=True)]
args += [track_options.model_copy(deep=True), visualize_options]
track.simultaneous_track(*args, output_directory=output_parent)

In [ ]:
# num_processes = int(0.75 * os.cpu_count())
# num_processes = os.cpu_count()
num_processes = 4
kwargs = {"initializer": parallel.initialize_process, "processes": num_processes}
with log.logging_listener(), multiprocessing.get_context("spawn").Pool(**kwargs) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        time.sleep(1)
        args = [i, time_interval, data_options.model_copy(deep=True), grid_options.copy()]
        args += [track_options.model_copy(deep=True), visualize_options]
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)

In [ ]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

In [ ]:
analysis_options = analyze.mcs.AnalysisOptions()
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent, analysis_options)

In [ ]:
figure_name = f"mcs_gridrad_{event_start.replace('-', '')}"
figure_options = visualize.option.horizontal_attribute_options(
    figure_name, style="presentation", attributes=["velocity", "offset"]
)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)